# 2. Quality control and denoising
## Import Data & Packages

In [2]:
# 1 - Import all packages
import IPython
import pandas as pd
from qiime2 import Visualization
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [3]:
# 2 - Set working directory --> change this so it matches your path to the project folder
os.chdir("/home/jovyan/MicrobiomeAnalysis_TummyTribe/")

# Verify that your wroking directory is the overall project folder (.../MicrobiomeAnalysis_TummyTribe)
print("Current working directory:", os.getcwd())

Current working directory: /home/jovyan/MicrobiomeAnalysis_TummyTribe


In [4]:
# 3 - Data directories
data_in = "data/raw"
data_out = "data/preprocessing"
results_dir = "results/preprocessing"

In [4]:
# 4 - Have a look at the sequencing data
! qiime tools peek $data_in/sequences-demux-paired.qza

UUID:        b4782ab7-550b-41f5-b906-ca2cda29ca9b
Type:        SampleData[PairedEndSequencesWithQuality]
Data format: SingleLanePerSamplePairedEndFastqDirFmt


## Quality Control

In [5]:
! qiime demux summarize \
    --i-data $data_in/sequences-demux-paired.qza \
    --o-visualization $results_dir/raw-QC.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: results/preprocessing/raw-QC.qzv


In [5]:
Visualization.load(f"{results_dir}/raw-QC.qzv")

<visualization: Visualization uuid: 88f9e06a-4fad-4400-a14d-323466ec9373>

## Denoising - Amplicon Sequence Variants

In [7]:
! qiime dada2 denoise-paired !help

Usage: qiime dada2 denoise-paired [OPTIONS]

  This method denoises paired-end sequences, dereplicates them, and filters
  chimeras.

Inputs:
  --i-demultiplexed-seqs ARTIFACT SampleData[PairedEndSequencesWithQuality]
                          The paired-end demultiplexed sequences to be
                          denoised.                                 [required]
Parameters:
  --p-trunc-len-f INTEGER Position at which forward read sequences should be
                          truncated due to decrease in quality. This truncates
                          the 3' end of the of the input sequences, which will
                          be the bases that were sequenced in the last cycles.
                          Reads that are shorter than this value will be
                          discarded. After this parameter is applied there
                          must still be at least a 12 nucleotide overlap
                          between the forward and reverse reads. If 0 is
            

In [8]:
! qiime dada2 denoise-paired \
    --i-demultiplexed-seqs $data_in/sequences-demux-paired.qza \
    --p-trunc-len-f 140 \
    --p-trunc-len-r 140 \
    --p-n-threads 3 \
    --o-table $data_out/dada2_table.qza \
    --o-representative-sequences $data_out/dada2_rep_seq.qza \
    --o-denoising-stats $data_out/dada2_stats.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: data/preprocessing/dada2_table.qza
Saved FeatureData[Sequence] to: data/preprocessing/dada2_rep_seq.qza
Saved SampleData[DADA2Stats] to: data/preprocessing/dada2_stats.qza


In [9]:
! qiime metadata tabulate \
    --m-input-file $data_out/dada2_stats.qza \
    --o-visualization $results_dir/dada2_stats.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: results/preprocessing/dada2_stats.qzv


In [10]:
Visualization.load(f"{results_dir}/dada2_stats.qzv")

<visualization: Visualization uuid: 384abd05-5545-48db-ac84-eafc6cae8516>

### Feature Table

In [13]:
! qiime feature-table summarize \
    --i-table $data_out/dada2_table.qza \
    --m-sample-metadata-file $data_in/metadata.tsv \
    --o-visualization $results_dir/dada2_table.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: results/preprocessing/dada2_table.qzv


In [12]:
Visualization.load(f"{results_dir}/dada2_table.qzv")

<visualization: Visualization uuid: d00aac93-8318-4029-9692-6b0cebaa1c6a>

### With trunc 150

In [ ]:
! qiime dada2 denoise-paired \
    --i-demultiplexed-seqs $data_in/sequences-demux-paired.qza \
    --p-trunc-len-f 150 \
    --p-trunc-len-r 150 \
    --p-n-threads 3 \
    --o-table $data_out/dada2_table_150.qza \
    --o-representative-sequences $data_out/dada2_rep_seq_150.qza \
    --o-denoising-stats $data_out/dada2_stats_150.qza

In [ ]:
! qiime metadata tabulate \
    --m-input-file $data_out/dada2_stats_150.qza \
    --o-visualization $results_dir/dada2_stats_150.qzv

In [ ]:
Visualization.load(f"{results_dir}/dada2_stats_150.qzv")

In [ ]:
! qiime feature-table summarize \
    --i-table $data_out/dada2_table_150.qza \
    --m-sample-metadata-file $data_in/metadata_150.tsv \
    --o-visualization $results_dir/dada2_table_150.qzv

In [ ]:
Visualization.load(f"{results_dir}/dada2_table_150.qzv")

### with trunc 160/130

In [ ]:
! qiime dada2 denoise-paired \
    --i-demultiplexed-seqs $data_in/sequences-demux-paired.qza \
    --p-trunc-len-f 160 \
    --p-trunc-len-r 130 \
    --p-n-threads 3 \
    --o-table $data_out/dada2_table_160_130.qza \
    --o-representative-sequences $data_out/dada2_rep_seq_160_130.qza \
    --o-denoising-stats $data_out/dada2_stats_160_130.qza

In [ ]:
! qiime metadata tabulate \
    --m-input-file $data_out/dada2_stats_160_130.qza \
    --o-visualization $results_dir/dada2_stats_160_130.qzv

In [ ]:
Visualization.load(f"{results_dir}/dada2_stats_160_130.qzv")

In [ ]:
! qiime feature-table summarize \
    --i-table $data_out/dada2_table_160_130.qza \
    --m-sample-metadata-file $data_in/metadata_160_130.tsv \
    --o-visualization $results_dir/dada2_table_160_130.qzv

In [ ]:
Visualization.load(f"{results_dir}/dada2_table_160_130.qzv")

### with trunc 135

In [ ]:
! qiime dada2 denoise-paired \
    --i-demultiplexed-seqs $data_in/sequences-demux-paired.qza \
    --p-trunc-len-f 135 \
    --p-trunc-len-r 135 \
    --p-n-threads 3 \
    --o-table $data_out/dada2_table_135.qza \
    --o-representative-sequences $data_out/dada2_rep_seq_135.qza \
    --o-denoising-stats $data_out/dada2_stats_135.qza

In [ ]:
! qiime metadata tabulate \
    --m-input-file $data_out/dada2_stats_135.qza \
    --o-visualization $results_dir/dada2_stats_135.qzv

In [ ]:
Visualization.load(f"{results_dir}/dada2_stats_135.qzv")

In [ ]:
! qiime feature-table summarize \
    --i-table $data_out/dada2_table_135.qza \
    --m-sample-metadata-file $data_in/metadata_135.tsv \
    --o-visualization $results_dir/dada2_table_135.qzv

In [ ]:
Visualization.load(f"{results_dir}/dada2_table_135.qzv")